In [1]:
import ipywidgets as widgets
from IPython.display import display
from datetime import date

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors


In [2]:
from_date = date(1900, 1, 1) 
to_date = date.today()
to_date = date(2021, 9, 7) 

portfolio_Vol = pd.DataFrame()

symbols = ['LVS','BBY','LEN','SPLK','VTRS','XPO','MOS','OLED','TTEC','WING']
# symbols = ['THO','PZZA']

# Build Portfolio prices, find oldest common date and remove rows with no data.           
for ind,symb in enumerate(symbols):
    tmp = yf.download(symb, from_date, to_date, interval="1wk")    
    tmp = tmp.loc[:, "Adj Close":"Adj Close"]              # Remove columns except Adj Close    
    tmp.rename(columns={"Adj Close":symb}, inplace=True)    # Rename columns
    
    # Sort values in descending order, NA values at the bottom
    tmp.sort_values(by='Date', ascending=True, na_position='last',inplace=True)     
    
    if ind==0:
        portfolio_Vol = tmp
        continue
    
    closest_date = max(tmp.index[0],portfolio_Vol.index[0])   # Determine closest date - the oldest date for which there is information for both Stocks
    tmp = tmp.loc[tmp.index >= closest_date]                  # Remove rows of data earlier than closest date
    portfolio_Vol = portfolio_Vol.loc[portfolio_Vol.index >= closest_date]
    portfolio_Vol = portfolio_Vol.join(tmp)                   # Merge dataframes now that they have the same indexes

# RESAMPLE to specific day if needed
# portfolio_Vol = portfolio_Vol.resample("W-THU").first()
print(portfolio_Vol.head())
# print(portfolio_Vol.tail())

# Portfolio Summary by Shares and initial capital
portf_Vol_summary = pd.DataFrame()
portf_Vol_summary.index = symbols
portf_Vol_summary['net-allocation'] = [-10000,-10000,-10000,-10000,10000,10000,10000,10000,10000,10000]
portf_Vol_summary['shares']=abs(portf_Vol_summary['net-allocation'])//portfolio_Vol.iloc[0]
portf_Vol_summary['shares'] = portf_Vol_summary['shares'].astype(int)
portf_Vol_summary['weighting'] = portf_Vol_summary['net-allocation']/sum(portf_Vol_summary['net-allocation'])
portf_Vol_summary['long/short'] = [-1,-1,-1,-1,1,1,1,1,1,1]
print(portf_Vol_summary.head(len(symbols)))

# Calculate and Replace columns of prices in portfolio with Returns
for col in portfolio_Vol.columns:
    portfolio_Vol[col] = (portfolio_Vol[col]/portfolio_Vol[col].shift(1))-1

# Remove first NaN row
portfolio_Vol = portfolio_Vol.iloc[1:]
portfolio_Vol

# Negate returns of assets which are shorted
for i in portfolio_Vol:
    portfolio_Vol[i]=portfolio_Vol[i]*portf_Vol_summary['long/short'].loc[i]
# Portfolio Covariance Matrix
covariance_matrix = np.cov(portfolio_Vol.values, rowvar=False)
print("\nCovariance Matrix:")
print(covariance_matrix)

# Calculate the standard deviations of each asset
std_deviations = np.sqrt(np.diag(covariance_matrix))

# Calculate the correlation matrix
corr_matrix = covariance_matrix / np.outer(std_deviations, std_deviations)
print("\nCorrelation Matrix:")
print(corr_matrix)

#Average correlation per asset
port_av_corr = []
for row in corr_matrix:
    port_av_corr.append(np.mean(row))
print("\nRow-wise averages:")
print(port_av_corr)

# Average portfolio correlation
print("\nPortfolio correlation:")
print(np.mean(np.array(port_av_corr)))

# Portfolio Variance
portf_variance=0
for row,i in enumerate(portf_Vol_summary['weighting']):    
    for col,j in enumerate(portf_Vol_summary['weighting']):
        val = i*j*covariance_matrix[row][col]
        portf_variance = portf_variance + val
portf_variance = portf_variance/2

print("\nPortfolio variance:")
print(portf_variance)
print("\nPortfolio standard dev.:")
portf_std = np.sqrt(portf_variance)
print(portf_std)
print("\nPortfolio standard dev. (Anual):")
portf_std_annual = np.sqrt(portf_variance)*np.sqrt(52)
print(portf_std_annual)

print("\nScenario Analysis:")
gross_sharpe = np.array([0.1,1,1.5,2])
scenario = np.array([gross_sharpe,gross_sharpe*portf_std_annual])
print(scenario)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


2004-12-13 00:00:00


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


2004-12-13 00:00:00
2012-04-16 00:00:00


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


2012-04-16 00:00:00
2012-04-16 00:00:00


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

2012-04-16 00:00:00
2012-04-16 00:00:00



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

2012-04-16 00:00:00
2015-06-08 00:00:00
                  LVS        BBY        LEN       SPLK       VTRS        XPO  \
Date                                                                           
2015-06-08  39.928192  25.615211  43.529350  68.919998  65.906036  16.414228   
2015-06-15  40.554939  26.353710  44.602222  73.190002  64.045998  16.441896   
2015-06-22  39.617897  25.614756  47.664959  70.379997  61.756718  15.819359   
2015-06-29  41.645210  25.260345  46.949707  69.720001  62.704628  15.387042   
2015-07-06  42.434486  25.584587  48.875370  71.839996  63.688297  15.338622   

                  MOS       OLED       TTEC       WING  
Date                                                    
2015-06-08  38.410629  51.923996  22.093801  21.950710  
2015-06-15  38.669334  52.562992  22.984276  21.053736  
2015-06-22  40.144016  50.955803  22.992676  20.271574  
2015-06-29  40.230251  48.912930  22.681845  20.465319  
2015-07-06  38.841816  46.492451  22.891865  24.670328  
